In [ ]:
import header
paths = header.setup_environment()
import os
#import sys
#import glob
#import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
#from obspy.core import read, Stream, UTCDateTime
from obspy.core import UTCDateTime
#import FDSNtools
#import wrappers
#import SDS
import libWellData as LLE
#import matplotlib
#matplotlib.use( 'tkagg' )
#%matplotlib inline
plot_each_file = False

# Parse lookuptable
lookuptableDF = LLE.removed_unnamed_columns(pd.read_csv(paths['lookuptable']))
lookuptableDF.to_csv('lookuptable_backup.csv')
lookuptableDF = lookuptableDF.sort_values(by=['starttime'])
lookuptableDF['miniseed'] = False
print(paths)

COMBINED_DIR = os.path.join(paths['CORRECTED'], 'combined')
if not os.path.isdir(COMBINED_DIR):
    os.makedirs(COMBINED_DIR)

transducersDF = LLE.removed_unnamed_columns(pd.read_csv(paths['transducersCSVfile']))
MSEED_DIR = os.path.join(paths['outdir'], 'miniseed')
#os.system(f"rm -rf {MSEED_DIR}/*")
for index, row in lookuptableDF.iterrows():
    filename = row['outputfile']  
    filepath = os.path.join(paths['CORRECTED'],filename)
    df = LLE.removed_unnamed_columns(pd.read_csv(filepath))
    df['datetime'] = [UTCDateTime(ts).datetime for ts in df['TIMESTAMP']]
    df['date'] = df['datetime'].dt.date
    unique_dates = df['date'].unique()
    num_days = len(unique_dates)
    print(filepath,' has length: ',len(df), ' and contains ',num_days, ' unique dates')

    if plot_each_file:
        N = len(df.columns) - 3
        if N>0:
            fig, axs = plt.subplots(N, 1)
            fig.suptitle(filename)
            for i, col in enumerate(df.columns[2:-1]):
                print(col)
                df.plot(x='datetime', y=col, ax = axs[i])
            fig.savefig(filename.replace('csv','.png'))

    YMD, folder, basename, ext = filename.split('.') # YMD is just upload date
    fs, instruments, blah = basename.split('_')
    fs, crap = fs.split('hz')
    YYYY = YMD[0:4]
    MM = YMD[4:6]
    DD = YMD[6:8]
    if 'Sec' in blah:
        secs, seqno = blah.split('Sec')
        fs = int(1/float(secs))
    elif 'Hz' in blah:
        fs, seqno = blah.split('Hz')
    elif 'Sensors' in blah:
        crap, seqno = blah.split('Sensors')


    for this_date in unique_dates:
        print(this_date)
        df2  = df[(df['date'] == this_date)]
        print(df2)
        
        combined_pklfile = f"{this_date}_{instruments}_{fs}Hz.pkl"
        print(f"{filename} -> {combined_pklfile}", '\n\n')
        combined_pklpath = os.path.join(COMBINED_DIR, combined_pklfile)
        if os.path.isfile(combined_pklpath):
            try:
                masterdf = pd.read_pickle(combined_pklpath)
                combineddf = pd.concat([masterdf, df2])
                print(combineddf)
                combineddf.to_pickle(combined_pklpath)
            except Exception as e:
                print(e)
        else:
            df2.to_pickle(combined_pklpath)
        


{'HOME': '/home/thompsong', 'Developer': '/home/thompsong/Developer', 'tremor_explorer_lib': '/home/thompsong/Developer/tremorExplorer/lib', 'src': '/home/thompsong/Developer/kitchensinkGT/PROJECTS/ROCKETSEIS/launchpad_erosion', 'work': '/home/thompsong/work', 'outdir': '/home/thompsong/work/PROJECTS/KSC_EROSION', 'CORRECTED': '/home/thompsong/work/PROJECTS/KSC_EROSION/corrected', 'lookuptable': '/home/thompsong/work/PROJECTS/KSC_EROSION/corrected/lookuptable.csv', 'transducersCSVfile': '/home/thompsong/work/PROJECTS/KSC_EROSION/transducer_metadata.csv', 'SDS_TOP': '/home/thompsong/work/PROJECTS/KSC_EROSION/SDS', 'DB_DIR': '/home/thompsong/work/PROJECTS/KSC_EROSION/RSAM'}
/home/thompsong/work/PROJECTS/KSC_EROSION/corrected/20220826.20hz.20hz_Sensors_20Hz0.csv  has length:  158601  and contains  1  unique dates
2022-07-21
                      TIMESTAMP  RECORD    1226423    2151692  \
0       2022-07-21 13:47:50.000       0 -84.073946 -47.182080   
1       2022-07-21 13:47:50.050      